In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
a_pos_5289 = "/content/drive/My Drive/npy File All/A.thaliana5289_pos.fasta-ENAC-Len-5.npy"
a_neg_5289 = "/content/drive/My Drive/npy File All/A.thaliana5289_neg.fasta-ENAC-Len-5.npy"
a_pos_1000 = "/content/drive/My Drive/npy File All/A.thaliana1000indep_pos-ENAC-Len-5.npy"
a_neg_1000 = "/content/drive/My Drive/npy File All/A.thaliana1000indep_neg.fasta-ENAC-5.npy"

a_x_train_pos = np.load(a_pos_5289) 
a_x_train_neg = np.load(a_neg_5289) 
a_x_test_pos = np.load(a_pos_1000) 
a_x_test_neg = np.load(a_neg_1000) 


b_pos_5289 =  "/content/drive/My Drive/npy File All/A.thaliana5289_pos.fasta-Kmer-1.npy"
b_neg_5289 = "/content/drive/My Drive/npy File All/A.thaliana5289_neg.fasta-Kmer-1.npy"
b_pos_1000 = "/content/drive/My Drive/npy File All/A.thaliana1000indep_pos-Kmer-1.npy"
b_neg_1000 = "/content/drive/My Drive/npy File All/A.thaliana1000indep_neg.fasta-kmer-1.npy"


b_x_train_pos = np.load(b_pos_5289) 
b_x_train_neg = np.load(b_neg_5289) 
b_x_test_pos = np.load(b_pos_1000) 
b_x_test_neg = np.load(b_neg_1000) 

y_train_pos = np.tile(1, 5289) 
y_train_neg = np.tile(0, 5289) 
y_test_pos = np.tile(1, 1000) 
y_test_neg = np.tile(0, 1000)

print(a_x_test_neg.shape)
print(a_x_train_neg.shape)
print(a_x_test_pos.shape)
print(a_x_train_pos.shape)
print(b_x_test_neg.shape)
print(b_x_train_neg.shape)
print(b_x_test_pos.shape)
print(b_x_train_pos.shape)

(1000, 148)
(5289, 148)
(1000, 148)
(5289, 148)
(1000, 4)
(5289, 4)
(1000, 4)
(5289, 4)


In [11]:
a_x_training = np.concatenate((a_x_train_pos, a_x_train_neg), axis = 0)
a_x_test = np.concatenate((a_x_test_pos, a_x_test_neg), axis = 0)

b_x_training = np.concatenate((b_x_train_pos, b_x_train_neg), axis = 0)
b_x_test = np.concatenate((b_x_test_pos, b_x_test_neg), axis = 0)

y_training = np.concatenate((y_train_pos,y_train_neg))
y_test = np.concatenate((y_test_pos,y_test_neg))

In [12]:
x_training = np.concatenate((a_x_training, b_x_training), axis = 1)

x_test = np.concatenate((a_x_test, b_x_test), axis = 1)

In [13]:
seed = 40
np.random.seed(seed)

kf = KFold(n_splits=5, random_state=seed, shuffle=True)

In [14]:

test_avg_acc_list = []
j = 1
for c in range(1,452,50):
  RFC = RandomForestClassifier(n_estimators=c,max_depth=None)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 1
Train Accuracy :  0.9105412432049161 	Validation Accuracy :  0.7542533081285444 	Test Accuracy  0.5845
Train Accuracy :  0.904632474592295 	Validation Accuracy :  0.7386578449905482 	Test Accuracy  0.593
Train Accuracy :  0.9145592058614984 	Validation Accuracy :  0.7651228733459358 	Test Accuracy  0.585
Train Accuracy :  0.9190594351884674 	Validation Accuracy :  0.7881796690307329 	Test Accuracy  0.579
Train Accuracy :  0.9201228878648233 	Validation Accuracy :  0.774468085106383 	Test Accuracy  0.6625
Avg Test Accuracy: 0.6008
Epoch 2/10 for C = 51
Train Accuracy :  1.0 	Validation Accuracy :  0.9494328922495274 	Test Accuracy  0.7255
Train Accuracy :  1.0 	Validation Accuracy :  0.9465973534971645 	Test Accuracy  0.7335
Train Accuracy :  1.0 	Validation Accuracy :  0.9607750472589792 	Test Accuracy  0.7195
Train Accuracy :  1.0 	Validation Accuracy :  0.9498817966903074 	Test Accuracy  0.7295
Train Accuracy :  1.0 	Validation Accuracy :  0.9536643026004729 	Tes

In [16]:
test_avg_acc_list = []
j = 1
for c in range(380,431,10):
  RFC = RandomForestClassifier(n_estimators=c,max_depth=None)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{5} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/5 for C = 380
Train Accuracy :  1.0 	Validation Accuracy :  0.9617202268431002 	Test Accuracy  0.7395
Train Accuracy :  1.0 	Validation Accuracy :  0.9532136105860114 	Test Accuracy  0.737
Train Accuracy :  1.0 	Validation Accuracy :  0.9697542533081286 	Test Accuracy  0.7435
Train Accuracy :  1.0 	Validation Accuracy :  0.9598108747044918 	Test Accuracy  0.725
Train Accuracy :  1.0 	Validation Accuracy :  0.9617021276595744 	Test Accuracy  0.737
Avg Test Accuracy: 0.7364
Epoch 2/5 for C = 390
Train Accuracy :  1.0 	Validation Accuracy :  0.9621928166351607 	Test Accuracy  0.7335
Train Accuracy :  1.0 	Validation Accuracy :  0.9517958412098299 	Test Accuracy  0.7425
Train Accuracy :  1.0 	Validation Accuracy :  0.9640831758034026 	Test Accuracy  0.7355
Train Accuracy :  1.0 	Validation Accuracy :  0.9598108747044918 	Test Accuracy  0.744
Train Accuracy :  1.0 	Validation Accuracy :  0.9645390070921985 	Test Accuracy  0.7445
Avg Test Accuracy: 0.74
Epoch 3/5 for C = 400
Train Ac

In [17]:
test_avg_acc_list = []
j = 1
for c in range(1,101,10):
  RFC = RandomForestClassifier(n_estimators=270,max_depth=c)
  val_accuracy_list = []
  train_accuracy_list = []
  test_acc_list = []
  test_avg = 0
  for train_index, val_index in kf.split(x_training):
    #print("train = ",train_index,"Val =",val_index)
    X_train, X_val = x_training[train_index], x_training[val_index]
    y_train, y_val = y_training[train_index], y_training[val_index]
    
    RFC.fit(X_train, y_train)

    y_val_pred = RFC.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_accuracy_list.append(val_accuracy)

    y_train_pred = RFC.predict(X_train)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    train_accuracy_list.append(train_accuracy)

    y_test_pred = RFC.predict(x_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_acc_list.append(test_accuracy)
    test_avg = test_avg + test_accuracy
  test_avg = test_avg/5
  print(f"Epoch {j}/{10} for C = {c}")
  j=j+1
  for i in range(0, len(train_accuracy_list)):
    print("Train Accuracy : ", train_accuracy_list[i], "\tValidation Accuracy : ", val_accuracy_list[i],"\tTest Accuracy ",test_acc_list[i])
  print("Avg Test Accuracy:",test_avg)

Epoch 1/10 for C = 1
Train Accuracy :  0.7988655164263767 	Validation Accuracy :  0.8024574669187146 	Test Accuracy  0.6365
Train Accuracy :  0.7650673599621839 	Validation Accuracy :  0.7632325141776938 	Test Accuracy  0.5985
Train Accuracy :  0.7823209643110376 	Validation Accuracy :  0.7764650283553876 	Test Accuracy  0.6155
Train Accuracy :  0.782228524164008 	Validation Accuracy :  0.7829787234042553 	Test Accuracy  0.597
Train Accuracy :  0.7784473590925204 	Validation Accuracy :  0.7725768321513002 	Test Accuracy  0.619
Avg Test Accuracy: 0.6133
Epoch 2/10 for C = 11
Train Accuracy :  0.9985818955329709 	Validation Accuracy :  0.9588846880907372 	Test Accuracy  0.7385
Train Accuracy :  0.998345544788466 	Validation Accuracy :  0.947069943289225 	Test Accuracy  0.7335
Train Accuracy :  0.9985818955329709 	Validation Accuracy :  0.9584120982986768 	Test Accuracy  0.7295
Train Accuracy :  0.9982275788727402 	Validation Accuracy :  0.9531914893617022 	Test Accuracy  0.7345
Train Acc